# AWS Glue Studio Notebook

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, hour, month, year, when, count, avg, sum, dayofmonth
from pyspark.sql.types import DoubleType
import pandas as pd
from pyspark.sql import functions as F
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 100)  

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: d5a60e9b-9327-4d27-a71f-348091751179
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session d5a60e9b-9327-4d27-a71f-348091751179 to get into ready status...
Session d5a60e9b-9327-4d27-a71f-348091751179 has been created.



In [2]:
spark = SparkSession.builder.appName("EVCharging").getOrCreate()

In [3]:
# Read from S3 bucket
data_uri = "s3://ev-charging-data/raw/ev_charging_patterns.csv"

In [4]:
df = spark.read.csv(data_uri, header=True, inferSchema=True)

In [5]:
df.limit(5).toPandas()

  User ID Vehicle Model  Battery Capacity (kWh) Charging Station ID Charging Station Location  \
0  User_1        BMW i3              108.463007         Station_391                   Houston   
1  User_2  Hyundai Kona              100.000000         Station_428             San Francisco   
2  User_3    Chevy Bolt               75.000000         Station_181             San Francisco   
3  User_4  Hyundai Kona               50.000000         Station_327                   Houston   
4  User_5  Hyundai Kona               50.000000         Station_108               Los Angeles   

  Charging Start Time   Charging End Time  Energy Consumed (kWh)  Charging Duration (hours)  \
0 2024-01-01 00:00:00 2024-01-01 00:39:00              60.712346                   0.591363   
1 2024-01-01 01:00:00 2024-01-01 03:01:00              12.339275                   3.133652   
2 2024-01-01 02:00:00 2024-01-01 04:48:00              19.128876                   2.452653   
3 2024-01-01 03:00:00 2024-01-01 06:4

In [6]:
df.printSchema()

root
 |-- User ID: string (nullable = true)
 |-- Vehicle Model: string (nullable = true)
 |-- Battery Capacity (kWh): double (nullable = true)
 |-- Charging Station ID: string (nullable = true)
 |-- Charging Station Location: string (nullable = true)
 |-- Charging Start Time: timestamp (nullable = true)
 |-- Charging End Time: timestamp (nullable = true)
 |-- Energy Consumed (kWh): double (nullable = true)
 |-- Charging Duration (hours): double (nullable = true)
 |-- Charging Rate (kW): double (nullable = true)
 |-- Charging Cost (USD): double (nullable = true)
 |-- Time of Day: string (nullable = true)
 |-- Day of Week: string (nullable = true)
 |-- State of Charge (Start %): double (nullable = true)
 |-- State of Charge (End %): double (nullable = true)
 |-- Distance Driven (since last charge) (km): double (nullable = true)
 |-- Temperature (°C): double (nullable = true)
 |-- Vehicle Age (years): double (nullable = true)
 |-- Charger Type: string (nullable = true)
 |-- User Type: str

In [7]:
df = df.drop("User ID")

In [8]:
# Check for missing values
missing_values = df.select([((count(when(col(c).isNull(), c)) / count("*")) * 100).alias(c) for c in df.columns])

In [9]:
missing_values.toPandas()

   Vehicle Model  Battery Capacity (kWh)  Charging Station ID  Charging Station Location  \
0            0.0                     0.0                  0.0                        0.0   

   Charging Start Time  Charging End Time  Energy Consumed (kWh)  Charging Duration (hours)  \
0                  0.0                0.0                    5.0                        0.0   

   Charging Rate (kW)  Charging Cost (USD)  Time of Day  Day of Week  State of Charge (Start %)  \
0                 5.0                  0.0          0.0          0.0                        0.0   

   State of Charge (End %)  Distance Driven (since last charge) (km)  Temperature (°C)  \
0                      0.0                                       5.0               0.0   

   Vehicle Age (years)  Charger Type  User Type  
0                  0.0           0.0        0.0


The dataset contains 5% missing values. We can choose to delete these values.

In [10]:
# Drop rows with missing values
df = df.dropna()

In [11]:
# Check for duplicates
duplicate_count = df.count() - df.dropDuplicates().count()
print(f"Number of duplicates: {duplicate_count}")

Number of duplicates: 0


In [12]:
# Calculate the actual charging duration in hours
df = df.withColumn("Computed Charging Duration (hours)", 
                   (col("Charging End Time").cast("long") - col("Charging Start Time").cast("long")) / 3600)

# Validate existing 'Charging Duration (hours)' against computed duration
df = df.withColumn("Duration Difference", 
                   col("Charging Duration (hours)") - col("Computed Charging Duration (hours)"))

In [13]:
# Check discrepancies
discrepancy_threshold = 0.1  # hours
discrepancies = df.filter(F.abs(col("Duration Difference")) > discrepancy_threshold)
print(f"Number of discrepancies: {discrepancies.count()}")

Number of discrepancies: 1070


In [14]:
# Update Charging Duration
df = df.withColumn("Charging Duration (hours)", col("Computed Charging Duration (hours)"))

In [15]:
# Extract hour and day
df = df.withColumn("Charging Start Hour", hour(col("Charging Start Time")))
df = df.withColumn("Charging Start Day", dayofmonth(col("Charging Start Time")))
df = df.withColumn("Charging End Hour", hour(col("Charging End Time")))
df = df.withColumn("Charging End Day", dayofmonth(col("Charging End Time")))

In [16]:
# Define peak hours
df = df.withColumn("Peak Charging", 
                   when((col("Charging Start Hour").between(7, 10)) | 
                        (col("Charging Start Hour").between(17, 20)), 
                        "Peak").otherwise("Off-Peak"))

In [17]:
# Battery Health Indicators
# Assuming 1.8% degradation per year --> https://www.geotab.com/blog/ev-battery-health/
df = df.withColumn("Effective Battery Capacity (kWh)", 
                   col("Battery Capacity (kWh)") * (1 - 0.018 * col("Vehicle Age (years)")))

In [18]:
df = df.drop("Battery Capacity (kWh)")

In [19]:
# Create the new feature "Battery Percentage Charged"
df = df.withColumn("Battery Percentage Charged", 
    col("State of Charge (End %)") - col("State of Charge (Start %)")
)

In [20]:
df = df.drop("State of Charge (End %)", "State of Charge (Start %)")

In [21]:
# Create the new feature "Temperature x Charging Duration"
df = df.withColumn("Temperature x Charging Duration", 
    col("Temperature (°C)") * col("Charging Duration (hours)")
)

In [22]:
# Create the new feature "Charging Rate x Battery Capacity"
df = df.withColumn("Charging Rate x Battery Capacity", 
    col("Charging Rate (kW)") * col("Effective Battery Capacity (kWh)")
)

In [23]:
# Calculate cost efficiency
df = df.withColumn("Cost per kWh (USD/kWh)", 
                   col("Charging Cost (USD)") / col("Energy Consumed (kWh)"))

In [24]:
# Function to convert a single column name to snake_case
def to_snake_case(col_name):
    return col_name.lower().replace(' ', '_')

old_columns = df.columns

new_columns = [to_snake_case(col) for col in old_columns]

for old_col, new_col in zip(old_columns, new_columns):
    df = df.withColumnRenamed(old_col, new_col)

In [25]:
df.limit(5).toPandas()

  vehicle_model charging_station_id charging_station_location charging_start_time  \
0        BMW i3         Station_391                   Houston 2024-01-01 00:00:00   
1  Hyundai Kona         Station_428             San Francisco 2024-01-01 01:00:00   
2    Chevy Bolt         Station_181             San Francisco 2024-01-01 02:00:00   
3  Hyundai Kona         Station_327                   Houston 2024-01-01 03:00:00   
4  Hyundai Kona         Station_108               Los Angeles 2024-01-01 04:00:00   

    charging_end_time  energy_consumed_(kwh)  charging_duration_(hours)  charging_rate_(kw)  \
0 2024-01-01 00:39:00              60.712346                   0.650000           36.389181   
1 2024-01-01 03:01:00              12.339275                   2.016667           30.677735   
2 2024-01-01 04:48:00              19.128876                   2.800000           27.513593   
3 2024-01-01 06:42:00              79.457824                   3.700000           32.882870   
4 2024-01-01 0

In [26]:
# Save file to S3
output_uri = "s3://ev-charging-processed/data/"
df.write.parquet(output_uri, mode="overwrite")